In [1]:
import torch

import pandas as pd
import numpy as np

# loading 60.000 rows of training data from MNIST
df_mnist = pd.read_csv(u'data/mnist_train.csv', sep=',', header=None)

# distributing the data as features & target
features, target = np.array(df_mnist.iloc[:, [i+1 for i in range(784)]]), np.array(df_mnist.iloc[:, 0])

In [2]:
# N is batch size, getting N rows in every iteration; D_in is input dimension 28x28=784;
# H is hidden dimension, we decide; D_out is output dimension, which is 1.
N, D_in, H, D_out = 1000, 784, 28, 10

# defining the functions to be used between our layers
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# defining the loss function
loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

# eta value of this regression
learning_rate = 0.005

# defining optimizer algorithm
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# defining epoch limit
max_epoch = 5000

# defining epoch step to print loss
print_epoch = 500

# starting epochs
for t in range(max_epoch):
    # selecting N rows for every epoch
    idx = np.random.choice(len(features), size=N, replace=True)
    
    # fetching the selected data as x & y
    x, y = torch.FloatTensor(features[idx]), torch.LongTensor(target[idx])
    
    # calculating y_pred by passing fetched x to our model
    y_pred = model(x)

    # comparing calculated y_pred and actual y
    loss = loss_fn(y_pred, y)
    
    # printing the loss in every print_epoch epochs and at the last
    if(t%print_epoch == 0 or t+1 == max_epoch):
        print(t, loss.item())

    # before the backward pass, zero all of the gradients for the variables it will update
    optimizer.zero_grad()

    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()

    # update optimizer parameters
    optimizer.step()

0 25406.67578125
500 477.96466064453125
1000 289.9354553222656
1500 231.67152404785156
2000 285.6107177734375
2500 211.6334686279297
3000 192.4965362548828
3500 265.86175537109375
4000 303.474609375
4500 173.66099548339844
4999 272.0728454589844


In [3]:
# now, our model is trained, it is time for 10.000 rows of test data
df_mnist_test = pd.read_csv(u'data/mnist_test.csv', sep=',', header=None)

# distributing the test data as x_test & y_test
x_test, y_test = torch.FloatTensor(np.array(df_mnist_test.iloc[:, [i+1 for i in range(784)]])), torch.LongTensor(np.array(df_mnist_test.iloc[:, 0]))

# getting the 10-layer results of the test data from the trained model
results = model(x_test)

# preparing a 1-layer result array
result = torch.zeros(len(results))

# parsing the maximum valued layer as our result for every row
for i in range(len(results)):
    result[i] = results[i].argmax()

print("Result:\n", result, "\n")

# building a confusion matrix
confusion_matrix = torch.zeros((10, 10))

# filling up the confusion matrix
for i in range(len(result)):
    confusion_matrix[int(y_test[i]), int(result[i])] += 1
    
print("Confusion matrix:\n", confusion_matrix, "\n")
    
# calculating model accuracy
accuracy = torch.sum(torch.diag(confusion_matrix))/torch.sum(confusion_matrix)

print("Accuracy:", accuracy*100, "%\n")

Result:
 tensor([7., 2., 1.,  ..., 4., 5., 6.]) 

Confusion matrix:
 tensor([[ 961.,    0.,    4.,    0.,    0.,    4.,    5.,    4.,    1.,    1.],
        [   0., 1064.,   14.,    2.,    0.,    0.,    4.,    8.,   43.,    0.],
        [   9.,   12.,  956.,    5.,    4.,    1.,   11.,   15.,   17.,    2.],
        [   3.,   16.,   46.,  882.,    0.,   16.,    2.,   13.,   19.,   13.],
        [   1.,    4.,   19.,    0.,  897.,    5.,   17.,    5.,    8.,   26.],
        [   6.,    1.,    4.,   49.,    2.,  755.,   13.,    4.,   51.,    7.],
        [  11.,    1.,   11.,    0.,    5.,   14.,  899.,    1.,   16.,    0.],
        [   4.,    9.,   27.,    1.,    5.,    2.,    0.,  946.,    1.,   33.],
        [   8.,    8.,   16.,    1.,    7.,   11.,    6.,    7.,  901.,    9.],
        [   4.,    3.,    4.,   11.,   31.,    8.,    0.,   22.,   11.,  915.]]) 

Accuracy: tensor(91.7600) %

